In [ ]:
# Client Setup
import boto3

# client = boto3.client("bedrock-runtime", region_name="us-west-2")
# model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"

session = boto3.Session(profile_name="bedrock-dev")
client = session.client("bedrock-runtime", region_name="us-west-2")


# Claude model ID — must be correct and available in the region
model_id = "us.anthropic.claude-3-sonnet-20240229-v1:0"

In [ ]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [ ]:
# Tool Schemas

article_details_schema = {
    "toolSpec": {
        "name": "article_details",
        "description": "This tool should be called with details about an article. It accepts information about the article's title, author, and related topics.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "The title of the article. Can be left empty.",
                    },
                    "author": {
                        "type": "string",
                        "description": "The name of the article's author. Can be left empty.",
                    },
                    "topics": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A list of topics or categories that the article covers. Can be an empty list.",
                    },
                },
            }
        },
    }
}

to_json_schema = {
    "toolSpec": {
        "name": "to_json",
        "description": "This tool processes any JSON data and can be used for generating structured content, transforming information, or creating any JSON-based output needed for your task.",
        "inputSchema": {
            "json": {"type": "object", "additionalProperties": True}
        },
    }
}

In [ ]:
messages = []

add_user_message(
    messages,
    "Write a one-paragraph scholarly article about computer science. Include a title and author name",
)

result = chat(messages)

add_assistant_message(messages, result["text"])

result["text"]